<a href="https://colab.research.google.com/github/tlsgptj/emotional_BERT/blob/main/KOBERT.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#크롤링
#기쁨, 슬픔, 놀람, 분노, 공포, 혐오
import json
import csv

with open('/content/감성대화말뭉치(최종데이터)_Training.json', 'r') as json_file:
  data = json.load(json_file)

with open('csv_file.csv', 'w', newline='', encoding='utf-8') as csv_file:
  writer = csv.writer(csv_file)
  header = data[0].keys()
  writer.writerow(header)
  for row in data:
    writer.writerow(row.values())

In [ ]:
!pip install mxnet # 코랩 환경이기 때문에 앞에 !를 붙여야 한다.
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers torch

In [ ]:
from transformers import AutoTokenizer, BertModel

# Try loading the tokenizer using AutoTokenizer
try:
    tokenizer = AutoTokenizer.from_pretrained('skt/kobert-base-v1')
except Exception as e:
    print(f"Error loading tokenizer: {e}")
    print("Please check the model name and your internet connection.")

# Load the KoBERT model
model = BertModel.from_pretrained('skt/kobert-base-v1')

# ... rest of your code ...le_with_warmup

In [ ]:
import json
import csv

# JSON 파일 로드
with open('/content/감성대화말뭉치(최종데이터)_Training.json', 'r', encoding='utf-8') as json_file:
    data = json.load(json_file)

# HS01과 HS02 추출 및 합치기
extracted_data = []
for row in data:
    talk_data = row.get('talk', {})  # 'talk' 데이터를 가져옴
    content = talk_data.get('content', {})
    hs01 = content.get('HS01', '')
    hs02 = content.get('HS02', '')

    # HS01과 HS02를 합치기
    combined_hs = f"{hs01} {hs02}".strip()  # 문자열을 합치고 불필요한 공백 제거

    if combined_hs:  # combined_hs가 비어있지 않으면 추가
        extracted_data.append([combined_hs])

# CSV 파일로 저장
csv_file_path = 'extracted_hs_data.csv'
with open(csv_file_path, mode='w', newline='', encoding='utf-8') as file:
    writer = csv.writer(file)
    writer.writerow(['Combined_HS'])  # 컬럼명 작성
    writer.writerows(extracted_data)

print(f'CSV 파일이 {csv_file_path}에 저장되었습니다.')

In [ ]:
# talk에 있는 데이터만 추출해야함

In [ ]:
import pandas as pd
emotional = pd.read_csv('/content/extracted_hs_data.csv')

In [ ]:
emotional.head(20)

In [ ]:
from transformers import AutoTokenizer, BertModel

# Try loading the tokenizer using AutoTokenizer
try:
    tokenizer = AutoTokenizer.from_pretrained('skt/kobert-base-v1')
except Exception as e:
    print(f"Error loading tokenizer: {e}")
    print("Please check the model name and your internet connection.")

# Load the KoBERT model
model = BertModel.from_pretrained('skt/kobert-base-v1')

# ... rest of your code ...le_with_warmup

In [ ]:
# 필요한 과정
# 1. 텍스트 데이터 정제
# 2. 문서를 토큰으로 나누기
# 3. 불용어 제거
# 4. 모델을 적용할꺼임 KOBERT적용 등...
# 5. TF-DIF로 알고리즘 비교
# koBERT의 경우 벡터로 변환할 필요가 없다고 함 : KoBERT는 텍스트를 바로 토큰화한 뒤 임베딩 벡터로 변환하는 기능을 내장하고 있기 때문

In [ ]:
# 전처리 과정 자세히
# [영어]
# 소문자 변환
# 불필요한 공백 제거
# 구두점 제거
# 특수 문자 제거
# 숫자 제거
# 불용어 제거
# 형태소 분석
# 어간 추출
# 표제어 추출
# 철자 교정
# 단어 토큰화
# 벡터화
## -> 여기서 한국어일 경우 1번은 생략하고 분석

In [ ]:
# 구두점 제거
import string
import re
emotional['Combined_HS'] = emotional['Combined_HS'].str.translate(str.maketrans('', '', string.punctuation))
# 특수 문자 제거
emotional['Combined_HS'] = emotional['Combined_HS'].apply(lambda x: re.sub(r"[^a-zA-Z0-9가-힣\s]", "", x) if isinstance(x, str) else x)
# 숫자 제거
emotional['Combined_HS'] = emotional['Combined_HS'].apply(lambda x: re.sub(r'\d+', '', x) if isinstance(x, str) else x)
# 불필요한 공백 제거를 해야하냐..?

In [ ]:
import nltk
from nltk.corpus import stopwords

nltk.download('stopwords')

# 한국어 불용어 추가
stop_words = set(stopwords.words('english'))
stop_words.update(['많아서','받아','나의','년','싶지','친구와','하네','시간이','자기','같은데','살','벌써','직장에서','결혼을','돼서','신경','조금','근데','친구들과','어떤','있지','꼭','친구에게','대해','가고','회사에','함께','돈도','나보다','이젠','빨리','친구들은','부모님께','하니','들어서','일도','아주','나랑','결혼','되었어','됐어','노후','모든','학교','할까','나서','몇','건강이','자식들이','하지만','그동안','먼저','학교에서','나와','중이야','많은','병원에','그런데','기분이야','있을','건지','나이','친한','받았어','많아', '뭘','되고','마음이','자주','점점','무슨','위해','있을까','제대로','사는','그래도','일','직장','된','친구를','아무것도','먹고','것을','모두','아무도','하나', '아빠가','참','없는데','말이야','때마다','될','어제','공부를','친구','해도','되는','전에', '하면','보면', '성적이','큰','또', '않고', '같은','것이', '없이','사람들이','얼마', '이번', '회사', '집에', '앞으로', '걸까','아직', '하지', '텐데', '매일', '부모님이', '자신이', '수가', '나에게', '생각이', '않아서', '이번에', '회사에서', '건', '그런', '그래서', '없고', '줄', '그래서', '나만', '없는', '했어', '이다', '하다', '때', '돼', '싶은데', '없어서', '날', '나이가', '같이', '난다', '것', '같아', '나는', '내가', '다', '계속', '나', '내', '더', '남편이', '아내가', '오늘', '난', '정말', '많이', '요즘', '이제', '친구가', '나도', '자꾸', '해', '좀', '나를', '했는데', '응', '그', '갑자기', '거야', '엄마가', '할수', '항상', '잘', '있는데', '어떻게 해야', '때문에', '아들이', '이제는', '친구들이', '딸이', '그렇게', '돈을', '다들', '다시', '않아', '그냥', '있어', '진짜', '돈', '너무', '안', '못', '계', '수', '하고', '없어', '할', '어떻게', '봐', '게', '하는', '우리', '왜', '거', '해서', '한', '있는', '들어', '싶어', '해야', '같아서', '하는데', '일을', '다른', '할지', '걸', '이', '이렇게', '일이', '말을', '것도', '몸이', '나한테', '지금', '있어서', '돈이', '보니', '이런', '사람이'])

def remove_stopwords(text):
  if isinstance(text, str):
    tokens = text.split()
    result = [word for word in tokens if not word in stop_words]
    return " ".join(result)
  else:
    return text

emotional['Combined_HS'] = emotional['Combined_HS'].apply(remove_stopwords)

In [ ]:
!apt-get update
!apt-get install -y fonts-nanum

In [ ]:
from collections import Counter

# Combine all text in the 'Combined_HS' column into a single string
text = ' '.join(emotional['Combined_HS'].astype(str).tolist())

# Split the string into words
words = text.split()

# Count the frequency of each word
word_counts = Counter(words)

# Print word counts
for word, count in word_counts.items():
    print(f"{word}: {count}")

In [ ]:
emotional.head(20)

In [ ]:
#